In [9]:
import torch
import numpy as np

### create dataset



In [10]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)



In [11]:
#data常用作变量名，我们将导入的data模块用Data代替
import torch.utils.data as Data
#将随机读取包含10个数据样本的小批量。
batch_size = 10

In [12]:
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

for X, y in data_iter:
    print(X, y)
    break
    

tensor([[-0.7356,  0.5688],
        [-0.4326, -0.1157],
        [-1.1894,  1.1363],
        [-0.3485, -0.8717],
        [ 0.4424,  1.3624],
        [-0.0561, -0.3276],
        [-0.6064,  2.1861],
        [ 1.2738,  0.8298],
        [-0.4800,  0.8546],
        [-1.7252,  0.7236]]) tensor([ 0.7864,  3.7374, -2.0251,  6.4619,  0.4445,  5.2209, -4.4298,  3.9352,
         0.3527, -1.7151])


### define model

In [ ]:
#nn的核心数据结构是Module
#如何用nn.Module实现一个线性回归模型


In [16]:
import torch.nn as nn

In [17]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构



LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [33]:
#Sequential是一个有序的容器，网络层将按照在传入Sequential的顺序依次被添加到计算图中。
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )
print(net)

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)


In [20]:
# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......
print(net)

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [21]:
# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))
print(net)



Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [22]:
#通过net.parameters()来查看模型所有的可学习参数，此函数将返回一个生成器。
for param in net.parameters():
    print(param)




Parameter containing:
tensor([[-0.1293, -0.1822]], requires_grad=True)
Parameter containing:
tensor([-0.6360], requires_grad=True)


### 3.3.4 初始化模型参数

In [23]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [ ]:
#如果这里的net是用3.3.3节一开始的代码自定义的，那么上面代码会报错，net[0].weight应改为net.linear.weight，bias亦然
# 因为net[0]这样根据下标访问子模块的写法只有当net是个ModuleList或者Sequential实例时才可以




In [30]:
loss = nn.MSELoss()

### 3.3.6 定义优化算法

In [24]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [27]:
net = LinearNet(num_inputs)
optimizer =optim.SGD([
                # 如果对某个参数不指定学习率，就使用最外层的默认学习率
                {'params': net.subnet1.parameters()}, #lr=0.03
                {'params': net.subnet2.parameters(), 'lr': 0.01}
            ], lr=0.03)

AttributeError: 'LinearNet' object has no attribute 'subnet1'

In [28]:
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

### 3.3.7 training method

In [34]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))




epoch 1, loss: 50.737408
epoch 2, loss: 33.391335
epoch 3, loss: 45.800076


In [35]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)


[2, -3.4] Parameter containing:
tensor([[-0.4062, -0.0958]], requires_grad=True)
4.2 Parameter containing:
tensor([-0.2276], requires_grad=True)
